# Instalando e importando utilidades necesarias

In [ ]:
#Instalando librerias necesarias para la conexion con yahoo finance
!pip install yfinance
!pip install yahoo_fin
!pip install requests-html
!pip install --upgrade yahoo_fin
!pip install matplotlib seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.9/143.9 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 12.4 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6049 sha256=0c3e62dbc9c9e3af7f866a588012285c4ae57263f31437a669b3e5c1edb8d947
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7


# Extracción de datos de opciones

In [ ]:
#Importando librerias
import datetime
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
from yahoo_fin import options
import yahoo_fin.stock_info as si

import seaborn as sns



#La siguiente función esta inspirada en el código que se encuentra en este url
# https://medium.com/@txlian13/webscrapping-options-data-with-python-and-yfinance-e4deb0124613
def options_chain(symbol):
    #Se crea objeto correspondiente al activo subyacente de interés
    tk = yf.Ticker(symbol)
    # Se obtiene las fechas de expiración de las opciones que no han expirado
    exps = tk.options
    # Se utiliza la fecha de hoy para usar en el historial
    underlying_price = tk.history(period='1d')['Close'][0]

    # información histórica para el activo subyacente
    # Se obtiene los datos de opciones para cada fecha de expiración posible
    options_df_list = []
    for e in exps:
        #Se obtiene un diccionario de las opciones que corresponden a la fecha de expiracion
        opt = tk.option_chain(e)
        opt = pd.concat([opt.calls,opt.puts])
        opt["expirationDate"] = e
        options_df_list.append(opt)
    options_data = pd.concat(options_df_list, ignore_index=True)
    #Se corrige fecha de expiración
    options_data["expirationDate"] = pd.to_datetime(
        options_data["expirationDate"]
    ) + datetime.timedelta(days=1)
    options_data["days_to_maturity"] = (
        options_data["expirationDate"] - datetime.datetime.today()
    ).dt.days
    options_data["dte"] =  options_data["days_to_maturity"]/365

    options_data["underlying_price"] = underlying_price
    options_data["CALL"] = options_data["contractSymbol"].str[4:].apply(lambda x: "C" in x)

    options_data[["bid", "ask", "strike", "volume", "impliedVolatility"]] = options_data[
        ["bid", "ask", "strike", "volume", "impliedVolatility"]
    ].apply(pd.to_numeric)
    options_data["mark"] = (
        options_data["bid"] + options_data["ask"]
    ) / 2

    # Calculate the midpoint of the bid-ask

    # Drop unnecessary and meaningless columns
    #options_data= options_data.drop(
    #    columns=[
    #        "contractSize",
    #        "currency",
    #        "change",
    #        "percentChange",
    #        "lastTradeDate",
    #        "lastPrice",
    #    ]
    #)

    return options_data



# Análisis exploratorio de datos

In [ ]:
#Extrayendo informacion de las opciones cuyo subyacente
sp500_options = options_chain("SPY")

In [ ]:
#dir(sp500_options)
sp500_options

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,expirationDate,days_to_maturity,dte,underlying_price,CALL,mark
0,SPY240628C00300000,2024-06-24 15:41:06+00:00,300.0,246.36,246.43,246.79,0.000000,0.00000,26.0,121.0,4.699223,True,REGULAR,USD,2024-06-29,0,0.000000,546.369995,True,246.610
1,SPY240628C00305000,2024-06-24 15:41:32+00:00,305.0,241.36,241.44,241.79,0.000000,0.00000,120.0,60.0,4.589848,True,REGULAR,USD,2024-06-29,0,0.000000,546.369995,True,241.615
2,SPY240628C00310000,2024-06-24 16:42:27+00:00,310.0,235.84,236.44,236.79,0.000000,0.00000,34.0,16.0,4.472661,True,REGULAR,USD,2024-06-29,0,0.000000,546.369995,True,236.615
3,SPY240628C00315000,2024-06-21 14:12:14+00:00,315.0,229.47,231.44,231.79,0.000000,0.00000,32.0,17.0,4.359380,True,REGULAR,USD,2024-06-29,0,0.000000,546.369995,True,231.615
4,SPY240628C00320000,2024-03-19 13:49:58+00:00,320.0,196.63,182.45,183.15,0.000000,0.00000,4.0,41.0,0.000010,True,REGULAR,USD,2024-06-29,0,0.000000,546.369995,True,182.800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6400,SPY261218P00760000,2024-03-01 16:00:48+00:00,760.0,249.30,234.00,239.00,0.000000,0.00000,1.0,0.0,0.240608,True,REGULAR,USD,2026-12-19,903,2.473973,546.369995,False,236.500
6401,SPY261218P00775000,2024-05-07 17:44:46+00:00,775.0,256.98,238.13,243.00,0.000000,0.00000,1.0,0.0,0.204041,True,REGULAR,USD,2026-12-19,903,2.473973,546.369995,False,240.565
6402,SPY261218P00780000,2024-05-06 13:43:18+00:00,780.0,265.98,243.23,248.00,0.000000,0.00000,1.0,0.0,0.206406,True,REGULAR,USD,2026-12-19,903,2.473973,546.369995,False,245.615
6403,SPY261218P00785000,2024-04-03 13:36:19+00:00,785.0,266.70,271.39,276.00,0.000000,0.00000,1.0,0.0,0.296043,True,REGULAR,USD,2026-12-19,903,2.473973,546.369995,False,273.695


In [ ]:
sp500_options.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6405 entries, 0 to 6404
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   contractSymbol     6405 non-null   object             
 1   lastTradeDate      6405 non-null   datetime64[ns, UTC]
 2   strike             6405 non-null   float64            
 3   lastPrice          6405 non-null   float64            
 4   bid                6405 non-null   float64            
 5   ask                6405 non-null   float64            
 6   change             6405 non-null   float64            
 7   percentChange      6405 non-null   float64            
 8   volume             6198 non-null   float64            
 9   openInterest       6403 non-null   float64            
 10  impliedVolatility  6405 non-null   float64            
 11  inTheMoney         6405 non-null   bool               
 12  contractSize       6405 non-null   object       

In [ ]:
sp500_options.describe()

,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,expirationDate,days_to_maturity,dte,underlying_price,mark
count,6405.000000,6405.000000,6405.000000,6405.000000,6405.000000,6405.000000,6198.000000,6403.000000,6405.000000,6405,6405.000000,6405.000000,6405.000000,6405.000000
mean,480.056831,49.293441,49.049875,49.811494,-0.007825,-2.579828,447.204259,2619.039669,0.306357,2025-01-08 23:12:06.744730624,193.966745,0.531416,546.369995,49.430685
min,120.000000,0.010000,0.000000,0.000000,-113.329994,-85.714290,1.000000,0.000000,0.000010,2024-06-29 00:00:00,0.000000,0.000000,546.369995,0.000000
25%,415.000000,1.090000,0.960000,1.000000,-0.030000,-1.960783,2.000000,21.000000,0.135949,2024-08-01 00:00:00,33.000000,0.090411,546.369995,0.980000
50%,495.000000,9.830000,9.120000,9.510000,0.000000,0.000000,5.000000,280.000000,0.210427,2024-10-19 00:00:00,112.000000,0.306849,546.369995,9.345000
75%,546.000000,66.610000,67.870000,69.400000,0.000000,0.000000,42.000000,1606.000000,0.346747,2025-03-22 00:00:00,266.000000,0.728767,546.369995,68.525000
max,820.000000,427.740000,427.690000,430.020000,46.009995,100.000000,125156.000000,213433.000000,4.699223,2026-12-19 00:00:00,903.000000,2.473973,546.369995,428.855000
std,117.616802,75.861937,76.187624,77.001784,1.867689,9.970815,4100.120435,9741.481284,0.358546,NaN,220.073784,0.602942,0.000000,76.590975


In [ ]:
price = si.get_live_price('SPY')
print(price)

546.3699951171875


**Análisis exploratorio de datos**

In [ ]:
sp500_options[['strike', 'lastPrice', 'volume', 'dte', 'underlying_price', 'CALL']].head()

,strike,lastPrice,volume,dte,underlying_price,CALL
0,300.0,246.36,26.0,0.0,546.369995,True
1,305.0,241.36,120.0,0.0,546.369995,True
2,310.0,235.84,34.0,0.0,546.369995,True
3,315.0,229.47,32.0,0.0,546.369995,True
4,320.0,196.63,4.0,0.0,546.369995,True


# Calibración de Black Scholes

In [ ]:
from scipy.stats import norm
import numpy as np

def black_scholes_call(S, X, T, r, sigma, eps=1e-8):
    # Esta funcion calcula el precio de una opcion europea de compra
    T = max(T,eps)
    sigma = max(sigma,eps)
    d1 = (np.log(S / X) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = S * norm.cdf(d1) - X * np.exp(-r * T) * norm.cdf(d2)
    return call_price

def black_scholes_put(S, X, T, r, sigma, eps=1e-8):
    #Esta funcion calcula el precio de una opcion europea de venta
    T = max(T,eps)
    sigma = max(sigma,eps)
    d1 = (np.log(S / X) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    put_price = X * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    return put_price


In [ ]:
from typing_extensions import dataclass_transform
def error_function(params, S0, data):
    sigma = params[0]
    total_error = 0
    #Para cada contrato se calcula el precio del modelo
    for index, row in data.iterrows():
        S = row["underlying_price"]
        K = row["strike"]
        T = row["dte"]
        r = 0.03959
        market_price = row["lastPrice"]
        option_type = row["CALL"]
        if option_type == True:
            model_price = black_scholes_call(S, K, T, r, sigma)
        else:
            model_price = black_scholes_put(S, K, T, r, sigma)

        #Se calcula el error cuadratico entre el precio de mercado y el del modelo
        total_error += (market_price - model_price)**2
    return total_error


In [ ]:
from scipy.optimize import minimize

S0= price
# Optimización

# Suposición inicial para la volatilidad
initial_guess = [0.2]

#Se busca la volatilidad que corresponde al minimo error cuadratico
result = minimize(error_function, initial_guess, args=(S0, sp500_options), bounds=[(0, None)])

# Volatilidad óptima
optimal_volatility = result.x[0]
print(f"Volatilidad óptima: {optimal_volatility:.2%}")


Volatilidad óptima: 17.74%


# Calibración Heston 1993

In [ ]:
#Este código se basa en el libro:
#Hilpisch, Y. (2015). Derivatives analytics with Python: data analysis, models, simulation, calibration and hedging. John Wiley & Sons.

from scipy.integrate import quad
import warnings

# Ignorando advertencias
warnings.filterwarnings('ignore')

options = sp500_options

def H93_char_func(u, T, r, kappa_v, theta_v, sigma_v, rho, v0):
    """Valuation of European call option in H93 model via Lewis (2001)
    Fourier-based approach: characteristic function.
    Parameter definitions see function BCC_call_value."""
    c1 = kappa_v * theta_v
    c2 = -np.sqrt(
        (rho * sigma_v * u * 1j - kappa_v) ** 2 - sigma_v**2 * (-u * 1j - u**2)
    )
    c3 = (kappa_v - rho * sigma_v * u * 1j + c2) / (
        kappa_v - rho * sigma_v * u * 1j - c2
    )
    H1 = r * u * 1j * T + (c1 / sigma_v**2) * (
        (kappa_v - rho * sigma_v * u * 1j + c2) * T
        - 2 * np.log((1 - c3 * np.exp(c2 * T)) / (1 - c3))
    )
    H2 = (
        (kappa_v - rho * sigma_v * u * 1j + c2)
        / sigma_v**2
        * ((1 - np.exp(c2 * T)) / (1 - c3 * np.exp(c2 * T)))
    )
    char_func_value = np.exp(H1 + H2 * v0)
    return char_func_value

def H93_int_func(u, S0, K, T, r, kappa_v, theta_v, sigma_v, rho, v0):
    """
    Fourier-based approach for Lewis (2001): Integration function.
    """
    char_func_value = H93_char_func(
        u - 1j * 0.5, T, r, kappa_v, theta_v, sigma_v, rho, v0
    )
    int_func_value = (
        1 / (u**2 + 0.25) * (np.exp(1j * u * np.log(S0 / K)) * char_func_value).real
    )
    return int_func_value

def H93_call_value(S0, K, T, r, kappa_v, theta_v, sigma_v, rho, v0):
    """Valuation of European call option in H93 model via Lewis (2001)

    Parameter definition:
    ==========
    S0: float
        initial stock/index level
    K: float
        strike price
    T: float
        time-to-maturity (for t=0)
    r: float
        constant risk-free short rate
    kappa_v: float
        mean-reversion factor
    theta_v: float
        long-run mean of variance
    sigma_v: float
        volatility of variance
    rho: float
        correlation between variance and stock/index level
    v0: float
        initial level of variance
    Returns
    =======
    call_value: float
        present value of European call option
    """
    int_value = quad(
        lambda u: H93_int_func(u, S0, K, T, r, kappa_v, theta_v, sigma_v, rho, v0),
        0,
        np.inf,
        limit=250,
    )[0]
    call_value = max(0, S0 - np.exp(-r * T) * np.sqrt(S0 * K) / np.pi * int_value)
    return call_value

import pandas as pd
from scipy.optimize import brute, fmin

i = 0
min_MSE = 500

def H93_error_function(p0):
    """Error function for parameter calibration via
    Lewis (2001) Fourier approach for Heston (1993).
    Parameters
    ==========
    kappa_v: float
        mean-reversion factor
    theta_v: float
        long-run mean of variance
    sigma_v: float
        volatility of variance
    rho: float
        correlation between variance and stock/index level
    v0: float
        initial, instantaneous variance
    Returns
    =======
    MSE: float
        mean squared error
    """
    global i, min_MSE
    kappa_v, theta_v, sigma_v, rho, v0 = p0
    if kappa_v < 0.0 or theta_v < 0.005 or sigma_v < 0.0 or rho < -1.0 or rho > 1.0:
        return 500.0
    if 2 * kappa_v * theta_v < sigma_v**2:
        return 500.0
    se = []
    for row, option in options.iterrows():
        model_value = H93_call_value(
            S0,
            option["strike"],
            option["dte"],
            0.03959,
            kappa_v,
            theta_v,
            sigma_v,
            rho,
            v0,
        )
        #Cuando no tenemos call option utilizamos put-call parity para calcular el precio del put
        if(option["CALL"]==False):
          model_value = model_value + option["strike"] * np.exp(-1 * 0.03959 * option["dte"]) - S0

        #Agregamos a la lista de errores cuadraticos
        se.append((model_value - option["lastPrice"]) ** 2)

    #MSE = sum(se) / len(se)
    MSE = np.mean(se)
    min_MSE = min(min_MSE, MSE)
    if i % 25 == 0:
        print("%4d |" % i, np.array(p0), "| %7.3f | %7.3f" % (MSE, min_MSE))
    i += 1
    return MSE

def H93_calibration_full():
    """Calibrates Heston (1993) stochastic volatility model to market quotes."""
    # First run with brute force
    # (scan sensible regions, for faster convergence)
    p0 = brute(
        H93_error_function,
        (
            (7.5, 10.6, 5.0),  # kappa_v
            (0.04, 0.041, 0.01),  # theta_v
            (0.25, 0.251, 0.1),  # sigma_v
            (0, 0.01, 0.25),  # rho
            (0.03, 0.031, 0.01),
        ),  # v0
        finish=None,
    )

    # Second run with local, convex minimization
    # (we dig deeper where promising results)
    opt = fmin(
        H93_error_function, p0, xtol=0.000001, ftol=0.000001, maxiter=750, maxfun=900
    )
    return {'p0':p0, 'opt':opt}

resultados_H93=H93_calibration_full()



   0 | [7.5  0.04 0.25 0.   0.03] | 217.657 | 217.657
  25 | [7.88323867e+00 3.06306374e-02 2.83662211e-01 2.27686991e-04
 3.02325825e-02] | 215.574 | 215.271
  50 | [ 9.28988610e+00  3.53711900e-02  3.26871915e-01 -9.23482363e-04
  1.24285604e-02] | 213.911 | 213.695
  75 | [ 9.91771152e+00  3.53142139e-02  3.70641209e-01 -1.63080801e-03
  3.70800683e-03] | 213.638 | 213.614
 100 | [ 8.84151788e+00  3.64108560e-02  7.86450639e-01 -8.96499676e-03
  2.87286182e-04] | 213.108 | 213.055


In [ ]:
resultados_H93

In [ ]:
options.head()